In [19]:
!pip install konlpy

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
import pandas as pd

Complaint = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Complaint.csv")
Complaint.tail(10)

,Subject,Content
1690,주택/도시계획/부동산,"수고 많으십니다.제 딸아이는 연세대학교에 재학중이고, 경남으로 주소지가 되어 있으며..."
1691,환경/공원/상수도,대기 측정소 좀 저희 동네에 설치해주시면 감사하겠습니다.그러면 저희 동네 대기 질 ...
1692,교통,시니어 패스 (어르신 교통카드)에도 버스 지하철 환승제도 적용해야 한다현 시니어 패...
1693,환경/공원/상수도,본인은 그린카라는 카쉐어링서비스를 이용했습니다처음 예약시 2014년 4월 19일 1...
1694,건강/식품위생,최근 서초구 우면동 7길 11 우면주공아파트에서 입주민이 아파트옥상에서 떨어져 자살...
1695,환경/공원/상수도,안녕하세요 원룸(7평)에서 거주하는 임대인인데 아무래도 공부 및 작업을 집에서 많이...
1696,여성/보육/아동 청소년,작년 15개구 시범운영을 거쳐 올해부터 25개 전 자치구에서 시행한다고 대대적으로 ...
1697,안전/소방/민방위,서울시 중구 다산로 165 신당중앙교회 앞 도로에서 정말 아주아주 멀쩡한 보도블럭을...
1698,교통,안녕하십니까이번 7월에 마곡지구에 입주하게 됨니다그간 서울시나 SH에서는 마곡나루역...
1699,환경/공원/상수도,인왕산로의 단군성전 앞부터 세검정 입구까지는 거주인은 전혀 없고이지역을 경비하는 경...


In [0]:
import math, sys
from konlpy.tag import Okt
import pickle
okt = Okt()

In [0]:
class Filter:
    def __init__(self):
        self.words = set() #어떤 단어들이 있는지. 집합
        self.word_dict = {} #이중 dictionary; [카테고리][단어] 가 몇번 사용됬는지. 히스토그램
        self.category_dict = {} #[카테고리] 가 몇번 사용됬는지
    
    ## text를 조사 어미 구두점을 제외한 단어만 list로 반환
    def split(self, text):
        results = []
        twitter = Okt() #형태소 분석기
        malist = twitter.pos(text, norm=True, stem=True) #steam True로 text를 분석.
        
        # 실습 2
        # 아래 for 문을 한줄짜리 for 문으로 바꿔보세요 List Comprehension
        # for word in malist:
        #     if not word[1] in ["Josa", "Eomi","Punctuation"]:
        #         results.append(word[0])
        
        # 조사, 어미, 구두점 이 아닌 word에 대한 word만 result에 저장.
        results = [word[0] for word in malist if not word[1] in ["Josa","Eomi","Punctuation"]]
                
                
        return results

    ## word_dict 히스토램(word_dict)과, word 목록에 추가하는 작업.
    def inc_word(self, word, category):
        if not category in self.word_dict:
            self.word_dict[category] = {}
        if not word in self.word_dict[category]:
            self.word_dict[category][word] = 0
        self.word_dict[category][word] += 1
        self.words.add(word)

    ## 카테고리 히스토그램 만들기.
    def inc_category(self, category):
        if not category in self.category_dict:
            self.category_dict[category] = 0
        self.category_dict[category] += 1

    ## 텍스트 넣어서 histogram 만들기
    def fit(self, text, category):
        word_list = self.split(text) ## 조사 어미 구두점 제외하여 list로 반환
        for word in word_list:
            self.inc_word(word,category)
        self.inc_category(category)
    
    ## score를 확률적 계산
    ## P(카테고리|전체문서) + P( 단어 | 해당카테고리)
    def score(self, words, category):
        score = math.log(self.category_prob(category)) #해당 카테고리가 나올 확률
        for word in words: # 각 단어에 대한 확률의 합.
            score += math.log(self.word_prob(word, category))
        return score

    def predict(self, text):
        best_category = None
        max_score = -sys.maxsize
        words = self.split(text) #형태소 분석 (조사 어미 구두점 빼고 단어 list)
        score_list = [] # [(카테고리,score) ...] 쌍으로 들어감. socre_list
        for category in self.category_dict.keys():
            score = self.score(words, category)
            score_list.append((category,score))
                        
            if score > max_score: #가장 높은 score와 카테고리를 저장.
                max_score = score
                best_category = category
                
        return best_category, score_list
    
    ## 해당 단어가, 카테고리에서 몇번이나 쓰였는지 가져오는 함수.
    def get_word_count(self, word, category):
        if word in self.word_dict[category]:
            return self.word_dict[category][word]
        else:
            return 0
    
    ## 전체 문서수에 대해 해당 카테고리가 몇번이나 나왔는지. 확률. #카테고리가 나올 확률
    def category_prob(self, category):
        sum_categories = sum(self.category_dict.values()) # 전체 문서의 숫자
        category_v = self.category_dict[category] # 해당 카테고리의 숫자
        return category_v / sum_categories # 카테고리 수 / 전체 문서의 수
    
    ## 
    def word_prob(self, word, category): # 
        n = self.get_word_count(word, category) + 1 # 해당 단어가 카테고리에서 몇번이나 쓰였는지. log(0)이 없으므로 +1 로 bias
        d = sum(self.word_dict[category].values()) + len(self.words) # 해당 카테고리의 단어의 수 + 전체 단어의 수
        return n/d

    def save_as_pickle(self,file_name='TF-IDF'):
      with open(file_name+'.pickle', 'wb') as f:
          pickle.dump(self, f)

In [0]:
bf = Filter()

for i in range(len(Complaint)):
  bf.fit(Complaint.Content[i], Complaint.Subject[i])

In [36]:
pre, scorelist = bf.predict('우리 집 앞 공원에 비둘기가 너무 많아요. 맨날 똥도 싸고 냄새나서 힘들어요. 어떻게 좀 해주세요')
print("결과 =", pre)
scorelist.sort(key=lambda element : element[1], reverse=True)
print(scorelist)

결과 = 환경/공원/상수도
[('환경/공원/상수도', -122.91701095948086), ('건강/식품위생', -124.75323769646354), ('교통', -132.84808445483532), ('안전/소방/민방위', -134.0885869693353), ('문화/관광/체육/디자인', -134.4985353667568), ('여성/보육/아동 청소년', -135.0709638705876), ('주택/도시계획/부동산', -136.7948939289482), ('기획/감사/교육/정보화/마을공동체', -137.12403775280933), ('복지/어르신/장애인', -138.41880180279392), ('경제/일자리', -143.49636728133984), ('건설', -145.0692566245729), ('세금/재정/계약', -149.07810871089598), ('기타', -155.66052633067108)]


In [0]:
bf.save_as_pickle('intent_model')

In [0]:
df = pickle.load(open('intent_model.pickle','rb'))

In [29]:
ttt = '우리 집 앞 공원에 비둘기가 너무 많아요. 맨날 똥도 싸고 냄새나서 힘들어요. 어떻게 좀 해주세요'
pre, scorelist = df.predict('우리 집 앞 공원에 비둘기가 너무 많아요. 맨날 똥도 싸고 냄새나서 힘들어요. 어떻게 좀 해주세요')
print("결과 =", pre)
print(scorelist)

결과 = 환경/공원/상수도
[('환경/공원/상수도', -122.91701095948086), ('문화/관광/체육/디자인', -134.4985353667568), ('교통', -132.84808445483532), ('경제/일자리', -143.49636728133984), ('안전/소방/민방위', -134.0885869693353), ('기획/감사/교육/정보화/마을공동체', -137.12403775280933), ('주택/도시계획/부동산', -136.7948939289482), ('건강/식품위생', -124.75323769646354), ('세금/재정/계약', -149.07810871089598), ('건설', -145.0692566245729), ('여성/보육/아동 청소년', -135.0709638705876), ('복지/어르신/장애인', -138.41880180279392), ('기타', -155.66052633067108)]
